In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/housing_fall2025


In [4]:
!pip install optuna xgboost lightgbm "mlflow<3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 136.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: cachetools
    Fou

In [2]:
base_folder = "/content/gdrive/MyDrive/Colab Notebooks/housing_fall2025"
%cd "{base_folder}"

/content/gdrive/MyDrive/Colab Notebooks/housing_fall2025


In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(f"{base_folder}/data/housing.db")
housing = pd.read_sql_query(
    """
    SELECT
        b.block_id,
        b.longitude,
        b.latitude,
        s.housing_median_age,
        s.total_rooms,
        s.total_bedrooms,
        s.population,
        s.households,
        s.median_income,
        s.median_house_value,
        op.name AS ocean_proximity
    FROM block AS b
    JOIN block_housing_stats AS s
        ON s.block_id = b.block_id
    JOIN ocean_proximity AS op
        ON op.ocean_proximity_id = b.ocean_proximity_id
    ORDER BY b.block_id
    """,
    conn,
)
conn.close()

housing.head()

,block_id,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,-122.23,37.88,41.0,880,129.0,322,126,8.3252,452600.0,NEAR BAY
1,1,-122.22,37.86,21.0,7099,1106.0,2401,1138,8.3014,358500.0,NEAR BAY
2,2,-122.24,37.85,52.0,1467,190.0,496,177,7.2574,352100.0,NEAR BAY
3,3,-122.25,37.85,52.0,1274,235.0,558,219,5.6431,341300.0,NEAR BAY
4,4,-122.25,37.85,52.0,1627,280.0,565,259,3.8462,342200.0,NEAR BAY


In [4]:
# =============================================================================
# FULL PIPELINE:
# - Build preprocessing
# - Stratified train/test split
# - Train & log 4 models WITHOUT PCA (Ridge, HGB, XGBoost, LightGBM)
# - Train & log 4 models WITH PCA (preprocessing + PCA(0.95) + model)
# - Pick GLOBAL best among 8 models by Test MAE
# - Save, load, and compare the global best model
# =============================================================================

import os
import numpy as np
import pandas as pd

from dotenv import load_dotenv

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor

import mlflow
from mlflow.models import infer_signature
import joblib

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# -----------------------------------------------------------------------------
# ASSUMPTION: housing DataFrame is already loaded.
# It must contain:
#   - median_income, median_house_value, block_id
#   - latitude, longitude
#   - total_bedrooms, total_rooms, households, population, median_income
#   - any categorical columns (object dtype)
#
# Example:
# housing = pd.read_csv("housing.csv")
# -----------------------------------------------------------------------------


# =============================================================================
# STEP 1: Build Full ML Preprocessing Pipeline
# =============================================================================

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(
            self.n_clusters,
            n_init=10,
            random_state=self.random_state
        )
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def fit_transform(self, X, y=None, sample_weight=None):
        self.fit(X, y, sample_weight)
        return self.transform(X)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]


def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler(),
    )

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler(),
)

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.0, random_state=42)

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"),
)

default_num_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler(),
)

preprocessing = ColumnTransformer(
    [
        ("bedrooms",        ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
        ("people_per_house",ratio_pipeline(), ["population", "households"]),
        ("log",             log_pipeline,
            ["total_bedrooms", "total_rooms", "population",
             "households", "median_income"]),
        ("geo",             cluster_simil, ["latitude", "longitude"]),
        ("cat",             cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=default_num_pipeline,
)

print("✓ STEP 1: Preprocessing pipeline created.")


# =============================================================================
# STEP 2: Split Data into Stratified Train and Test Sets
# =============================================================================

housing["income_cat"] = pd.cut(
    housing["median_income"],
    bins=[0, 1.5, 3.0, 4.5, 6, np.inf],
    labels=[1, 2, 3, 4, 5],
)

train_set, test_set = train_test_split(
    housing,
    test_size=0.20,
    stratify=housing["income_cat"],
    random_state=42,
)

# Drop income_cat after stratification
for df in (train_set, test_set):
    df.drop("income_cat", axis=1, inplace=True)

X_train = train_set.drop(["block_id", "median_house_value"], axis=1).copy()
y_train = train_set["median_house_value"].copy()

X_test = test_set.drop(["block_id", "median_house_value"], axis=1).copy()
y_test = test_set["median_house_value"].copy()

print(f"✓ STEP 2: Stratified split done. Train size: {len(X_train)}, Test size: {len(X_test)}")


# =============================================================================
# STEP 3: Define 4 Model Pipelines (WITHOUT PCA)
# =============================================================================

ridge_pipeline = make_pipeline(
    preprocessing,
    Ridge()
)

hgb_pipeline = make_pipeline(
    preprocessing,
    HistGradientBoostingRegressor(random_state=42)
)

xgb_pipeline = make_pipeline(
    preprocessing,
    XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="hist",
        n_jobs=-1,
    ),
)

lgbm_pipeline = make_pipeline(
    preprocessing,
    LGBMRegressor(
        random_state=42,
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=-1,
    ),
)

models = {
    "ridge": ridge_pipeline,
    "histgradientboosting": hgb_pipeline,
    "xgboost": xgb_pipeline,
    "lightgbm": lgbm_pipeline,
}

print("✓ STEP 3: 4 baseline model pipelines defined.")


# =============================================================================
# STEP 4: Configure MLflow (e.g., Dagshub) via .env
# =============================================================================

load_dotenv()  # .env should contain MLFLOW_TRACKING_URI, USERNAME, PASSWORD

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

if MLFLOW_TRACKING_USERNAME:
    os.environ["MLFLOW_TRACKING_USERNAME"] = MLFLOW_TRACKING_USERNAME
if MLFLOW_TRACKING_PASSWORD:
    os.environ["MLFLOW_TRACKING_PASSWORD"] = MLFLOW_TRACKING_PASSWORD

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("median_house_pricing_multi_model")

print("✓ STEP 4: MLflow configured.")


# =============================================================================
# STEP 5: Train, Evaluate, and Log 4 Baseline Models (NO PCA)
# =============================================================================

results = {}  # name -> {"pipeline": ..., "test_mae": ...}

for name, pipeline in models.items():
    print(f"\n{'=' * 80}")
    print(f"Training baseline model: {name}")
    print(f"{'=' * 80}")

    # Fit
    pipeline.fit(X_train, y_train)

    # Evaluate
    y_pred = pipeline.predict(X_test)
    test_mae = mean_absolute_error(y_test, y_pred)
    print(f"{name} (no PCA) Test MAE: ${test_mae:,.2f}")

    results[name] = {"pipeline": pipeline, "test_mae": test_mae}

    # Log to MLflow
    with mlflow.start_run(run_name=f"{name}_baseline"):
        mlflow.log_param("model_family", name)
        mlflow.log_param("uses_pca", False)

        # Log estimator params
        est_step_name = list(pipeline.named_steps.keys())[-1]
        est = pipeline.named_steps[est_step_name]
        est_params = {f"{est_step_name}__{k}": v for k, v in est.get_params().items()}
        mlflow.log_params(est_params)

        # Metric
        mlflow.log_metric("test_MAE", test_mae)

        # Signature + model
        signature = infer_signature(X_train, pipeline.predict(X_train))
        mlflow.sklearn.log_model(
            sk_model=pipeline,
            artifact_path="housing_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{name}_pipeline",
        )

print("\n✓ STEP 5: All 4 baseline models trained and logged.")


# =============================================================================
# STEP 6: Helper to Build an Estimator by Name (for PCA variants)
# =============================================================================

def make_estimator_for_name(name: str):
    if name == "ridge":
        return Ridge()
    elif name == "histgradientboosting":
        return HistGradientBoostingRegressor(random_state=42)
    elif name == "xgboost":
        return XGBRegressor(
            objective="reg:squarederror",
            random_state=42,
            n_estimators=300,
            learning_rate=0.1,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            tree_method="hist",
            n_jobs=-1,
        )
    elif name == "lightgbm":
        return LGBMRegressor(
            random_state=42,
            n_estimators=300,
            learning_rate=0.05,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            n_jobs=-1,
        )
    else:
        raise ValueError(f"Unknown model name: {name}")


# =============================================================================
# STEP 7: Train, Evaluate, and Log PCA Versions of ALL 4 Models
# =============================================================================

pca_results = {}  # key: "<name>_with_pca" -> {"pipeline": ..., "test_mae": ...}

for name in models.keys():
    print("\n" + "=" * 80)
    print(f"Training PCA-augmented model: {name}")
    print("=" * 80)

    est = make_estimator_for_name(name)

    # Pipeline: preprocessing -> PCA -> estimator
    pca_pipeline = make_pipeline(
        preprocessing,
        PCA(n_components=0.95),
        est,
    )

    pca_pipeline.fit(X_train, y_train)
    y_pred_pca = pca_pipeline.predict(X_test)
    test_mae_pca = mean_absolute_error(y_test, y_pred_pca)

    model_key = f"{name}_with_pca"
    pca_results[model_key] = {
        "pipeline": pca_pipeline,
        "test_mae": test_mae_pca,
    }

    print(f"{model_key} Test MAE: ${test_mae_pca:,.2f}")

    # Log PCA model to MLflow
    with mlflow.start_run(run_name=model_key):
        mlflow.log_param("model_family", name)
        mlflow.log_param("uses_pca", True)

        # Estimator params
        est_step_name = list(pca_pipeline.named_steps.keys())[-1]
        est_step = pca_pipeline.named_steps[est_step_name]
        est_params = {f"{est_step_name}__{k}": v for k, v in est_step.get_params().items()}
        mlflow.log_params(est_params)

        # PCA params
        pca_step = pca_pipeline.named_steps["pca"]
        mlflow.log_param("pca__n_components", pca_step.n_components)

        # Metric
        mlflow.log_metric("test_MAE", test_mae_pca)

        # Signature + model
        signature_pca = infer_signature(X_train, pca_pipeline.predict(X_train))
        mlflow.sklearn.log_model(
            sk_model=pca_pipeline,
            artifact_path="housing_model_with_pca",
            signature=signature_pca,
            input_example=X_train,
            registered_model_name=f"{name}_pipeline_with_pca",
        )

print("\n✓ STEP 7: All 4 PCA models trained and logged.")


# =============================================================================
# STEP 8: Choose GLOBAL Best Model (with or without PCA)
# =============================================================================

# Combine baseline and PCA results
all_results = {}
all_results.update(results)      # "ridge", "xgboost", ...
all_results.update(pca_results)  # "ridge_with_pca", "xgboost_with_pca", ...

global_best_name = min(all_results, key=lambda k: all_results[k]["test_mae"])
global_best_mae = all_results[global_best_name]["test_mae"]
global_best_pipeline = all_results[global_best_name]["pipeline"]

uses_pca = "with_pca" in global_best_name

print("\n" + "=" * 80)
print("GLOBAL BEST MODEL (ACROSS 8 CANDIDATES)")
print("=" * 80)
print(f"Global best model key: {global_best_name}")
print(f"Global best Test MAE:  ${global_best_mae:,.2f}")
print(f"Uses PCA:               {uses_pca}")


# =============================================================================
# STEP 9: Save, Load, and Compare the GLOBAL Best Model
# =============================================================================

def save_model(model, filename="global_best_model.pkl"):
    joblib.dump(model, filename)
    print(f"✓ Model saved to {filename}")

def load_model(filename="global_best_model.pkl"):
    model = joblib.load(filename)
    print(f"✓ Model loaded from {filename}")
    return model

def compare_model_predictions(model_in_memory, model_loaded, X, y, n_samples=5):
    print("\n" + "=" * 80)
    print("MODEL COMPARISON: In-Memory vs Loaded from Disk")
    print("=" * 80)

    preds_mem = model_in_memory.predict(X)
    preds_load = model_loaded.predict(X)

    mae_mem = mean_absolute_error(y, preds_mem)
    mae_load = mean_absolute_error(y, preds_load)

    print(f"\nTest MAE (memory): ${mae_mem:,.2f}")
    print(f"Test MAE (loaded): ${mae_load:,.2f}")
    print(f"Difference in MAE:  ${abs(mae_mem - mae_load):,.2f}")

    print(f"\n{'-' * 80}")
    print(f"Sample Predictions (first {n_samples}):")
    print(f"{'-' * 80}")
    print(f"{'Actual':<15} {'Memory Model':<15} {'Loaded Model':<15} {'Diff':<15}")
    print(f"{'-' * 80}")

    for i in range(min(n_samples, len(y))):
        actual = y.iloc[i] if hasattr(y, "iloc") else y[i]
        pm = preds_mem[i]
        pl = preds_load[i]
        diff = abs(pm - pl)
        print(f"${actual:<14,.0f} ${pm:<14,.2f} ${pl:<14,.2f} ${diff:<14,.2f}")

    diff_all = np.abs(preds_mem - preds_load)
    are_identical = np.allclose(preds_mem, preds_load)
    max_diff = diff_all.max()

    print(f"\n{'-' * 80}")
    if are_identical:
        print("✓ SUCCESS: Predictions are identical (within numerical precision)")
    else:
        print("✗ WARNING: Predictions differ!")
    print(f"Maximum absolute difference: {max_diff:.10f}")
    print(f"Total predictions compared: {len(preds_mem)}")
    print("=" * 80)

    return {
        "mae_memory": mae_mem,
        "mae_loaded": mae_load,
        "are_identical": are_identical,
        "max_difference": max_diff,
    }


print("\n" + "-" * 80)
print("Saving and reloading GLOBAL best model...")
print("-" * 80)

save_model(global_best_pipeline, filename=f"{base_folder}/models/global_best_model.pkl")
loaded_global_best = load_model(f"{base_folder}/models/global_best_model.pkl")

# Remove. Just for demo purposes

comparison = compare_model_predictions(
    model_in_memory=global_best_pipeline,
    model_loaded=loaded_global_best,
    X=X_test,
    y=y_test,
    n_samples=10,
)

print("\nDone:")
print(f"- GLOBAL best model key: {global_best_name}")
print(f"- GLOBAL best Test MAE:  ${global_best_mae:,.2f}")
print(f"- Saved & loaded global best; predictions match: {comparison['are_identical']}")

2025/12/10 10:56:36 INFO mlflow.tracking.fluent: Experiment with name 'median_house_pricing_multi_model' does not exist. Creating a new experiment.


✓ STEP 1: Preprocessing pipeline created.
✓ STEP 2: Stratified split done. Train size: 16512, Test size: 4128
✓ STEP 3: 4 baseline model pipelines defined.
✓ STEP 4: MLflow configured.

Training baseline model: ridge
ridge (no PCA) Test MAE: $52,350.18


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'ridge_pipeline'.
Created version '1' of model 'ridge_pipeline'.



Training baseline model: histgradientboosting
histgradientboosting (no PCA) Test MAE: $30,702.41


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Successfully registered model 'histgradientboosting_pipeline'.
Created version '1' of model 'histgradientboosting_pipeline'.


...


Training PCA-augmented model: lightgbm
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002270 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 9
[LightGBM] [Info] Start training from score 206333.518653


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


lightgbm_with_pca Test MAE: $37,631.80


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valid


✓ STEP 7: All 4 PCA models trained and logged.

GLOBAL BEST MODEL (ACROSS 8 CANDIDATES)
Global best model key: xgboost
Global best Test MAE:  $28,465.49
Uses PCA:               False

--------------------------------------------------------------------------------
Saving and reloading GLOBAL best model...
--------------------------------------------------------------------------------
✓ Model saved to /content/gdrive/MyDrive/Colab Notebooks/housing_fall2025/models/global_best_model.pkl
✓ Model loaded from /content/gdrive/MyDrive/Colab Notebooks/housing_fall2025/models/global_best_model.pkl

MODEL COMPARISON: In-Memory vs Loaded from Disk

Test MAE (memory): $28,465.49
Test MAE (loaded): $28,465.49
Difference in MAE:  $0.00

--------------------------------------------------------------------------------
Sample Predictions (first 10):
--------------------------------------------------------------------------------
Actual          Memory Model    Loaded Model    Diff           
--------